In [1]:
import pandas as pd
import psycopg2
import numpy as np
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
from supabase import create_client, Client
import supabase

In [2]:
# Load csv into a dataframe
data = pd.read_csv('13-07-2022-14_59_00_master.csv')   
df = pd.DataFrame(data)

In [3]:
# Clean data
df['clarity_level'] = df['clarity_level'].fillna('none') # fill NaN values with 'none'
df['bad_egg'] = df['bad_egg'].astype(str) # change bad_egg column from boolean to string
df['clarity_level'] = np.where(df['bad_egg'] == 'True', df['bad_egg'], df['clarity_level']) # combine bad_egg and clarity_level where bad_egg = True
df['clarity_level'] = df['clarity_level'].replace(['True'],'bad') # change value of bad_egg from 'True' to 'bad'
df = df.drop(['bad_egg', 'split'], axis=1) # drop bad_egg and split columns
df.head()

,label,youtube_id,time_start,time_end,clarity_level
0,crowd,4pRPc04xoEo1,0,12,none
1,spray,4pRPc04xoEo2,12,18,hard
2,running,4pRPc04xoEo3,12,21,hard
3,crowd,4pRPc04xoEo4,34,44,none
4,crowd,4pRPc04xoEo5,66,71,none


In [4]:
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
client = create_client(url, key)

email = "jackson.shinpaugh@gmail.com"
password = "Zestymeatloaf1"
user = client.auth.sign_in(email=email, password=password)



In [5]:
urls = []

for ind, row in df.iterrows():

    id = str(row['youtube_id'])
    url = f'https://www.youtube.com/watch?v={id[0:11]}'
    urls.append(url)

unique_urls = set(urls)

In [6]:
count = 1

for url in unique_urls:
    try:
        youtube_id = url[32:]
        query = client.table("urls").insert({"id": count, "url": url, "youtube_id": youtube_id}).execute()
        count = count + 1
    except:
        continue

In [42]:
query = client.table("users").insert({"id":"1", "first_name": "Jackson", "last_name": "Shinpaugh", 
                                    "email": "jackson.shinpaugh@gmail.com", "password": "Zestymeatloaf1"}).execute()

In [21]:
#establishing the connection
conn = psycopg2.connect(
   database="var_data", user='postgres', password='Zestymeatloaf1', host='localhost', port= '5432'
)
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [14]:
query ='''CREATE TABLE var_data(
    LABEL text,
    YOUTUBE_ID varchar(20),
    CLIP_NUMBER int,
    TIME_START int,
    TIME_END int,
    CLARITY_LEVEL text,
    USER_ID int)'''

cursor.execute(query)

In [22]:
def insert_into_table(cursor, label, youtube_id, clip_number, time_start, time_end, clarity, user_id):
    insert_into_var_data = ("""INSERT INTO var_data (label, youtube_id, clip_number, time_start,
                        time_end, clarity_level, user_id)
    VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (label, youtube_id, clip_number, time_start, time_end, clarity, user_id)
    cursor.execute(insert_into_var_data, row_to_insert)

In [43]:
for ind, row in df.iterrows():
    
    try:
        id = row['youtube_id']
        youtube_id = row['youtube_id'][0:11]
        
        query = client.table("var_data").insert({"id":id, "label": row['label'], "time_start": row['time_start'], 
                                                "time_end": row['time_end'], "clarity_level": row['clarity_level'],
                                                "youtube_id": youtube_id, "user_id": 1}).execute()
    
    except:
        continue

In [24]:
# Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()